In [10]:
import os
import shutil
import sh

In [11]:
path = "file:///home/kadochnikov/data/air_travel/"
filename = "200*.csv"
csv_file = path+filename


In [12]:
travel_df= sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true', delimiter=',', quote='"').\
load(csv_file).cache()

In [13]:
travel_df.first()

Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime=u'1232', CRSDepTime=1225, ArrTime=u'1341', CRSArrTime=1340, UniqueCarrier=u'WN', FlightNum=2891, TailNum=u'N351', ActualElapsedTime=u'69', CRSElapsedTime=u'75', AirTime=u'54', ArrDelay=u'1', DepDelay=u'7', Origin=u'SMF', Dest=u'ONT', Distance=389, TaxiIn=u'4', TaxiOut=u'11', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=u'0', WeatherDelay=u'0', NASDelay=u'0', SecurityDelay=u'0', LateAircraftDelay=u'0')

In [15]:
travel_df.printSchema()


root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [17]:
travel_df.select('ArrDelay', 'DepDelay', 'Origin', 'Dest').show(5)

+--------+--------+------+----+
|ArrDelay|DepDelay|Origin|Dest|
+--------+--------+------+----+
|       1|       7|   SMF| ONT|
|       8|      13|   SMF| PDX|
|      34|      36|   SMF| PDX|
|      26|      30|   SMF| PDX|
|      -3|       1|   SMF| PDX|
+--------+--------+------+----+
only showing top 5 rows



In [22]:
travel_df.createOrReplaceTempView("travel")

In [23]:
spark.sql("show tables").show(5)

+--------------------+-----------+
|           tableName|isTemporary|
+--------------------+-----------+
|                 a04|      false|
|           answer7v2|      false|
|   assign02_caseinfo|      false|
|assign03_bx_book_...|      false|
|         assignment2|      false|
|       assignment3_1|      false|
|       assignment3_2|      false|
|       assignment3_3|      false|
|              austin|      false|
|            biketrip|      false|
|        book_details|      false|
|     book_rate_stats|      false|
|         book_rating|      false|
|        book_ratings|      false|
| book_ratings_output|      false|
|          book_users|      false|
|         bookratings|      false|
|      books_ratings2|      false|
|        booksratings|      false|
|            bookstbl|      false|
|           bookusers|      false|
|                  bx|      false|
|            bx_books|      false|
|       bxuserstodrop|      false|
|            caseload|      false|
|         ccount_nk0

In [24]:
sqlContext.sql("\
select Year, count (*) as Flight_Cnt,\
avg(distance) as Distance_AVG \
from travel \
group by Year \
order by Year \
limit 10").show()

+----+----------+-----------------+
|Year|Flight_Cnt|     Distance_AVG|
+----+----------+-----------------+
|2007|   7453215|719.8057890453986|
|2008|   7009728|726.3870294253928|
+----+----------+-----------------+



In [38]:
travel_arrdelay = sqlContext.sql("\
select Origin, Dest, sum(ArrDelay) as Arrival_Delay \
from travel \
group by Origin, Dest \
order by Arrival_Delay desc")

In [39]:
travel_arrdelay.show(5)

+------+----+-------------+
|Origin|Dest|Arrival_Delay|
+------+----+-------------+
|   ORD| LGA|     460205.0|
|   LGA| ORD|     412650.0|
|   ORD| EWR|     398200.0|
|   EWR| ORD|     358300.0|
|   ATL| LGA|     354532.0|
+------+----+-------------+
only showing top 5 rows



In [40]:
travel_arrdelay_avg = sqlContext.sql("\
select Origin, Dest, avg(ArrDelay) as Arrival_Delay_Avg \
from travel \
group by Origin, Dest \
order by Arrival_Delay_Avg desc")

In [45]:
travel_arrdelay_avg.show(5)

+------+----+-----------------+
|Origin|Dest|Arrival_Delay_Avg|
+------+----+-----------------+
|   CMI| SPI|            575.0|
|   ONT| IAD|            370.0|
|   ELP| MFE|            316.0|
|   BHM| JFK|            252.0|
|   ACY| MYR|            252.0|
+------+----+-----------------+
only showing top 5 rows



In [36]:
travel_depdelay = sqlContext.sql("\
select Origin, Dest, sum(DepDelay) as Dep_Delay \
from travel \
group by Origin, Dest \
order by Dep_Delay desc")

In [37]:
travel_depdelay.show(5)

+------+----+---------+
|Origin|Dest|Dep_Delay|
+------+----+---------+
|   ORD| LGA| 458339.0|
|   ORD| EWR| 447232.0|
|   LGA| ORD| 379487.0|
|   LAX| SFO| 357331.0|
|   ATL| EWR| 356454.0|
+------+----+---------+
only showing top 5 rows



In [42]:
travel_depdelay_avg = sqlContext.sql("\
select Origin, Dest, avg(DepDelay) as Dep_Delay_Avg \
from travel \
group by Origin, Dest \
order by Dep_Delay_Avg desc")

In [43]:
travel_depdelay_avg.show(5)

+------+----+-------------+
|Origin|Dest|Dep_Delay_Avg|
+------+----+-------------+
|   CMI| SPI|        587.0|
|   ONT| IAD|        386.0|
|   ABQ| GJT|        366.0|
|   SDF| SPI|        329.0|
|   ELP| MFE|        307.0|
+------+----+-------------+
only showing top 5 rows



In [46]:
travel_sum = sqlContext.sql("\
select Origin, Dest, count(ArrDelay) as AD_count\
from travel \
group by Origin, Dest \
order by AD_count")

ParseException: u"\nextraneous input 'travel' expecting {<EOF>, ',', 'FROM', 'WHERE', 'GROUP', 'ORDER', 'HAVING', 'LIMIT', 'LATERAL', 'WINDOW', 'UNION', 'EXCEPT', 'INTERSECT', 'SORT', 'CLUSTER', 'DISTRIBUTE'}(line 1, pos 53)\n\n== SQL ==\nselect Origin, Dest, count(ArrDelay) as AD_countfrom travel group by Origin, Dest order by AD_count\n-----------------------------------------------------^^^\n"